In [88]:
import json
instance = 'medium_1'
input_file = f'/Users/user/Hackatons/KIRO-2025/KIRO-2024/{instance}.json'

with open(input_file) as f:
    data = json.load(f)

In [89]:
print(data.keys())
for key in data.keys():
    print(key+" : "+str(len(data[key])))

dict_keys(['parameters', 'constraints', 'vehicles', 'shops'])
parameters : 2
constraints : 30
vehicles : 297
shops : 3


In [90]:
print(data['vehicles'][2]['type'])

regular


In [91]:
import json
import numpy as np

def parse_input_data(input_file):
    with open(input_file) as f:
        data = json.load(f)
    return data

class Problem():
    def __init__(self, input_file):
        # Parse the input data and initialize various attributes
        self.data = parse_input_data(input_file)
        self.N_vehicles = len(self.get('vehicles'))  # Number of wind turbines
        self.N_shops = len(self.get('shops'))  # Number of substations
        self.store_to_id, self.id_to_store = self.get_different_indexes()
        self.lot_of_change_costs = self.get_lot_of_change_costs()
        self.rolling_window_costs = self.get_rolling_window_costs()
        self.batch_size_costs = self.get_batch_size_costs()
        self.two_tone_cars = self.get_two_tones_cars()
        self.delta = self.get("parameters").get("two_tone_delta")

    def get_two_tones_cars(self):
        two_tone_cars = [
            e['id'] - 1 # ids starts with 0, only for this, because I will be using dhia's function
            for e in  self.get('vehicles')
            if e['type'] == 'two-tone'
        ]
        # print('two_tone_cars' , two_tone_cars)
        return np.array(two_tone_cars) if two_tone_cars  else None

    
    def get_different_indexes(self) :   # I return the indexes of each store dict : name --> id, and id --> name
        store_to_id = {}
        id_to_store = {}
        shops_infos = self.get('shops')
        
        for shop_id, shop_info in enumerate(shops_infos):
            store_name = shop_info['name']
            store_to_id[store_name] = shop_id
            id_to_store[shop_id] = store_name

        return store_to_id, id_to_store


    def get_lot_of_change_costs(self):
         # Returns a list of tuples with the following structure:
        # (int id of the shop, cost, list of partitions as numpy arrays)

        costs = [
            (self.store_to_id[e["shop"]], e["cost"] , [np.array(p)-1 for p in e["partition"]]) # ids starts with 0
            for e in self.get('constraints') 
            if e['type'] == "lot_change"
        ]
        return costs
    
    def get_rolling_window_costs(self) :
        # Returns a list of tuples with the following structure:
        # (int id of the shop, cost, window size, max vehicles, set of vehicles as a numpy array)
        costs = [
            (self.store_to_id[e["shop"]], e["cost"], e["window_size"], e["max_vehicles"], np.array(e["vehicles"])-1) # ids starts with 0
            for e in self.get('constraints') 
            if e['type'] == "rolling_window"
        ]
        return costs
    
    def get_batch_size_costs(self) :
        # Returns a list of tuples with the following structure:
        # (int id of the shop, cost, min vehicles, max vehicles, list of partitions as numpy arrays)

        costs = [
            (self.store_to_id[e["shop"]], e["cost"] , e["min_vehicles"], e['max_vehicles'] , np.array(e["vehicles"])-1) # ids starts with 0
            for e in self.get('constraints') 
            if e['type'] == "batch_size"
        ]
        return costs
    
    def json_overview(self):
        # Print the total number of elements for each key in the data
        print("TOTAL NUMBERS")
        for key in data.keys():
            print("\t" + key + " : " + str(len(data[key])))

    def get(self, key, view=False):
        # Retrieve data based on the given key, optionally print it
        if view:
            print(data[key])
        return data[key]



In [92]:
# Example usage of the Problem class
problem_instance = Problem(input_file)

# Print each attribute
print("Parsed Data:")
print(problem_instance.data)

print("\nNumber of Vehicles:")
print(problem_instance.N_vehicles)

print("\nNumber of Shops:")
print(problem_instance.N_shops)

print("\nStore to ID Mapping:")
print(problem_instance.store_to_id)

print("\nID to Store Mapping:")
print(problem_instance.id_to_store)

print("\nLot of Change Costs:")
print(problem_instance.lot_of_change_costs)

print("\nRolling Window Costs:")
print(problem_instance.rolling_window_costs)

print("\nBatch Size Costs:")
print(problem_instance.batch_size_costs)

print("\nTwo Tones Vehicles:")
print(problem_instance.two_tone_cars)

print("Delta")
print(problem_instance.delta)

# Rolling window :
permutation = np.array([1, 3, 5, 4, 2, 6])
w_r = 4  # Window size = 4

print('The window of ', permutation, 'of size', (len(permutation), w_r - 1))

result = np.lib.stride_tricks.sliding_window_view(permutation, w_r - 1)
print('Is ', result)


Parsed Data:
{'parameters': {'two_tone_delta': 115, 'resequencing_cost': 10000}, 'constraints': [{'max_vehicles': 2, 'cost': 5000, 'id': 1, 'window_size': 3, 'vehicles': [4, 5, 8, 9, 11, 16, 18, 23, 26, 29, 30, 33, 34, 36, 37, 38, 41, 42, 45, 46, 49, 50, 53, 55, 64, 70, 76, 77, 79, 84, 85, 86, 89, 90, 91, 93, 94, 96, 97, 100, 103, 104, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 122, 123, 125, 127, 128, 134, 135, 138, 141, 144, 146, 148, 149, 150, 152, 154, 155, 157, 158, 159, 165, 166, 170, 171, 178, 179, 180, 181, 184, 188, 190, 191, 192, 195, 196, 197, 199, 201, 203, 204, 205, 211, 213, 215, 216, 217, 218, 219, 220, 223, 225, 226, 227, 230, 231, 233, 235, 237, 239, 240, 241, 243, 250, 253, 255, 256, 257, 258, 259, 265, 266, 268, 269, 272, 274, 276, 277, 279, 280, 281, 283, 285, 286, 291, 293, 294, 297], 'type': 'rolling_window', 'shop': 'assembly'}, {'max_vehicles': 5, 'cost': 2000, 'id': 2, 'window_size': 9, 'vehicles': [4, 5, 8, 9, 11, 16, 18, 23, 29, 30, 33, 34, 38, 42, 46,

In [93]:
import numpy as np
import random as rd
from itertools import product
from copy import deepcopy

def apply_cycle(sigma, ui, delta):
        '''
        sigma: current vehicles arangement (just before applying the new cycle).
        ui   : the current two-tone car to be moved back.
        '''
        n = len(sigma)
        ti = sigma.index(ui)
        
        shift = min( (n-1 - ti) , delta )   # delta tilde       
        
        # Apply cycle (ti, ti + shift, ti + shift-1, ... , ti + 1) on sigma
        sigma[ti : ti+shift] = sigma[ti+1 : ti+shift+1]  # ti + 1 -> ti + 2 -> .. -> ti + shift
        sigma[ti + shift] = ui                           # ti + shift -> ti
        return sigma

class solution():
    def __init__(self , problem_instance , construct = True ):
        self.problem_instance = problem_instance
        self.body = None   # Permutation of vehicles entering the body shop: body[i] is the ID of the vehicle in position i
        self.assembly = None   # Permutation of vehicles entering the assembly shop: assembly[i] is the ID of the vehicle in position i
        self.paint_entry = None  # Permutation of vehicles entering the paint shop: paint_entry[i] is the ID of the vehicle in position i
        self.two_tone_cars = self.problem_instance.two_tone_cars
        self.costs = {}      # dict in which I saved the values of each loss for debuging
        if construct :
            self.random_heuristic_construction()


        
        # Inverse permutations will be computed and stored here
        self.inverse_body = None
        self.inverse_assembly = None
        self.inverse_paint_entry = None
        self.inverse_paint_exit = None
        if construct :
            self.compute_all_inverses()

    def parse_solution(self, path) :
        solution = parse_input_data(path)
        assert solution["body"]["exit"] == solution["body"]["entry"]  # theses are lists
        assert solution["assembly"]["exit"] == solution["assembly"]["entry"]

        self.body = np.array(solution["body"]["entry"])-1 
        self.paint_entry = np.array(solution["paint"]["entry"])-1
        self.paint_exit = np.array(solution["paint"]["exit"])-1
        self.assembly = np.array(solution["assembly"]["entry"])-1

        assert list(self.paint_exit) == list(self.get_paint_exit(self.paint_entry))

        self.compute_all_inverses()

        
        

    def random_heuristic_construction(self) :
        self.body = np.random.permutation(self.problem_instance.N_vehicles)  # Random permutation for the body shop
        self.assembly = np.random.permutation(self.problem_instance.N_vehicles)  # Random permutation for the assembly shop
        self.paint_entry = np.random.permutation(self.problem_instance.N_vehicles)  # Random permutation for the paint shop

        if self.two_tone_cars.any() :
            self.get_paint_exit() # Permutation of vehicles exiting the paint shop: paint_exit[i] is the ID of the vehicle in position i


  
    def get_paint_exit(self):
        ''' 
        Get paint exit given: 
        - paint entry cars list. 
        - two-tone cars list (in any order).
        - delta.
        '''
        delta = self.problem_instance.delta
        two_tone_cars = list( self.two_tone_cars + 1) # Back to id 1 at first item
        paint_entry = list(self.paint_entry+1)      # v1,  ...   ,vn
        
        paint_exit = deepcopy(paint_entry)      # v1,  ...   ,vn
    
        two_tone_entry_pos = sorted([paint_entry.index(ui) for ui in two_tone_cars])    # t_u1 < ... < t_uk              
        
        for t_ui in two_tone_entry_pos:                # t_u1, ... , t_uk
            ui = paint_entry[t_ui]
            paint_exit = apply_cycle(paint_exit, ui, delta)

        self.paint_exit =  np.array(paint_exit)-1


    def compute_inverse_permutation(self, perm):
        """
        Computes the inverse of a given permutation.
        """
        inv_perm = np.empty_like(perm)
        inv_perm[perm] = np.arange(len(perm))
        
        return inv_perm

    def compute_all_inverses(self):
        if self.body.any():
            self.inverse_body = self.compute_inverse_permutation(self.body)
        if self.assembly.any():
            self.inverse_assembly = self.compute_inverse_permutation(self.assembly)
        if self.paint_entry.any():
            self.inverse_paint_entry = self.compute_inverse_permutation(self.paint_entry)
        if self.paint_exit.any():
            self.inverse_paint_exit = self.compute_inverse_permutation(self.paint_exit)
 

    def get_resequencing_cost(self) :
        cs = self.problem_instance.get('parameters')["resequencing_cost"]
        delta_body = self.problem_instance.get('shops')[self.problem_instance.store_to_id['body']]["resequencing_lag"]
        cost = 0
        body_to_paint_entry_cost = np.sum(cs * np.maximum(0, self.inverse_body - delta_body - self.inverse_paint_entry))
        cost += body_to_paint_entry_cost
        
        delta_paint = self.problem_instance.get('shops')[self.problem_instance.store_to_id['paint']]["resequencing_lag"]
        paint_exit_to_assembly_cost = np.sum(cs * np.maximum(0, self.inverse_paint_exit - delta_paint - self.inverse_assembly))
        cost += paint_exit_to_assembly_cost

        return cost
    
    def get_lot_change_constraints(self) :
        cost = 0 
        lot_of_changes_costs = self.problem_instance.lot_of_change_costs
        id_to_permutation = {0: self.body, 1: self.assembly, 2: self.paint_entry}

        for element in lot_of_changes_costs :
            id_store, c_l, list_of_partitions = element
            permutation = id_to_permutation[id_store]

            # for partition_l_before_one_hot in list_of_partitions:
            #     # Use numpy's built-in function to avoid creating partition_l multiple times
            #     partition_l = np.zeros(self.problem_instance.N_vehicles)
            #     partition_l[partition_l_before_one_hot] = 1  # One-hot encoding

            #     # Calculate the difference count (using slicing and efficient numpy operations)
            #     diff_count = np.count_nonzero(partition_l[partition_indices[:-1]] != partition_l[partition_indices[1:]])

            #     # Accumulate the cost
            #     cost += c_l * diff_count

            # Create a partition matrix for all one-hot encoded partitions at once
            partition_matrix = np.zeros((len(list_of_partitions), self.problem_instance.N_vehicles))
            for i, partition_indices_one_hot in enumerate(list_of_partitions):
                partition_matrix[i, partition_indices_one_hot] = 1  # Fill one-hot rows

            # Compare adjacent indices for all partitions at once
            diff_matrix = partition_matrix[:, permutation[:-1]] != partition_matrix[:,permutation[1:]]
            # cost += c_l*diff_matrix.sum() # Sum differences for each partition
            cost += c_l*np.count_nonzero(diff_matrix)


        return cost
    

    def get_rolling_window_constraints(self) :
        cost = 0
        rolling_window_costs = self.problem_instance.rolling_window_costs
        id_to_permutation = {0: self.body, 1: self.assembly, 2: self.paint_entry}

        for element in rolling_window_costs :
            id_store,c_r, w_r, M_r, vehicles_in_V_r = element
            permutation = id_to_permutation[id_store]
            
            vehicles_in_V_r_one_hot = np.zeros(self.problem_instance.N_vehicles)
            vehicles_in_V_r_one_hot[vehicles_in_V_r] = 1  # One-hot encoding

            # for t in range(1,N_vehicles - w_r + 1) :
            #     sub_cost = (np.maximum(0 ,vehicles_in_V_r_one_hot[permutation[t:t+w_r-1]] - M_r))
            #     cost+= c_r*sub_cost**2
                
            rolling_window = np.lib.stride_tricks.sliding_window_view(permutation, w_r - 1)

            # Sum the one-hot encoded vehicle counts for each window
            window_sums = np.sum(vehicles_in_V_r_one_hot[rolling_window], axis=1)

            sub_costs = np.maximum(0, window_sums - M_r) ** 2

            cost += c_r*np.sum(sub_costs)


        return cost
    
    def get_batch_size_constraints(self) :
        cost = 0
        batch_size_costs = self.problem_instance.batch_size_costs
        id_to_permutation = {0: self.body, 1: self.assembly, 2: self.paint_entry}

        for element in batch_size_costs :
            id_store,c_b, m_b, M_b, vehicles_in_V_b = element
            permutation = id_to_permutation[id_store]
            vehicles_in_V_b_one_hot = np.zeros(self.problem_instance.N_vehicles)
            vehicles_in_V_b_one_hot[vehicles_in_V_b] = 1  # One-hot encoding


            # time = np.arange(len(permutation)-1)
            # time_tilde = np.arange(len(permutation))
            # time_combination = time[:, None]- time_tilde[:, None]
            # interval_values = np.maximum( m_b- time_combination, time_combination - M_b)
            # gamma_matrix = np.maximum(0 , interval_values)**2
            
            time_indices = np.arange(len(permutation))
            time_differences = time_indices[:, None] - time_indices  + 1 # Compute pairwise time differences

            # Compute the interval values 
            interval_bounds = np.maximum(m_b - time_differences, time_differences - M_b)

            # gamma matrix (squared positive values of interval bounds)
            gamma_matrix = np.maximum(0, interval_bounds) ** 2

            bool_vector_1 = np.zeros(self.problem_instance.N_vehicles)
            bool_vector_2 = np.zeros(self.problem_instance.N_vehicles)
            bool_matrix_3 = np.zeros_like(time_differences)

            bool_vector_1[1:] = 1- vehicles_in_V_b_one_hot[permutation[:-1]]
            bool_vector_2[:-1] = 1 - vehicles_in_V_b_one_hot[permutation[1:]]


            one_hot_permutation = vehicles_in_V_b_one_hot[permutation]

            # Compute the sums using vectorized operations

            cumsum_x = np.cumsum(one_hot_permutation) # the matrix of M[t, t'] = sum(sigma[t]...sigma[t']) and then compare with == (t'-t+1) to see if they verify all the equality
            cumsum_x_padded = np.concatenate(([0], cumsum_x))
            M = cumsum_x[None,:] - cumsum_x_padded[:-1,None]

            M = np.triu(M)

            bool_matrix_3 = (M == interval_bounds)            

            # the product for the intersection !
            bool_matrix = bool_vector_1[:, None]*bool_vector_2[None, :]*bool_matrix_3

            # To be sure we take only into account t < N-1 :
            bool_matrix[-1,:] = 0 
            cost += c_b*np.sum(gamma_matrix*bool_matrix)

        return cost

    def get_cost(self):
            cost_final = 0
            self.costs = {}
            resequencing_cost = self.get_resequencing_cost()
            self.costs['resequencing_cost'] = resequencing_cost
            cost_final += resequencing_cost

            lot_change_constraints = self.get_lot_change_constraints()
            self.costs['lot_change_constraints'] = lot_change_constraints
            cost_final += lot_change_constraints

            rolling_window_constraints = self.get_rolling_window_constraints()
            self.costs['rolling_window_constraints'] = rolling_window_constraints
            cost_final += rolling_window_constraints

            batch_size_constraints = self.get_batch_size_constraints()
            self.costs['batch_size_constraints'] = batch_size_constraints
            cost_final += batch_size_constraints

            return cost_final
    def solution_project_format(self):
        solution = dict()
        solution["body"]["entry"] = list(self.body+1)
        solution["body"]["exit"] = list(self.body+1) # do for all
        solution["paint"]["entry"] = list(self.paint_entry+1)
        solution["paint"]["exit"] = list(self.paint_exit+1)
        solution["assembly"]["entry"] = list(self.assembly+1)
        solution["assembly"]["exit"] = list(self.assembly+1)
        return solution


In [94]:
problem_instance = Problem(input_file)
sol = solution(problem_instance , True)
# sol.parse_solution(f'/Users/user/Hackatons/KIRO-2025/KIRO-2024/{instance}_sol_dhia_rd.json')

x = sol.get_cost()
print("Total Cost:", x )  # Print the total cost

# Print all the relevant attributes
# print("Problem Instance:", sol.problem_instance)
# print("Body Permutation:", sol.body)
# print("Assembly Permutation:", sol.assembly)
# print("Paint Entry Permutation:", sol.paint_entry)
# print("Paint Exit Permutation:", sol.paint_exit)

# print("\nInverse Permutations:")
# print("Inverse Body:", sol.inverse_body)
# print("Inverse Assembly:", sol.inverse_assembly)
# print("Inverse Paint Entry:", sol.inverse_paint_entry)
# print("Inverse Paint Exit:", sol.inverse_paint_exit)

Total Cost: 294208800.0


In [95]:
# Print all the relevant attributes

# print("Resequencing Cost:", sol.get_resequencing_cost())  # Print the resequencing cost
# print("Lot Change Constraints:", sol.get_lot_change_constraints())  # Print the lot change constraints
# print("Rolling Window Constraints:", sol.get_rolling_window_constraints())  # Print the rolling window constraints
# print("Batch Size Constraints:", sol.get_batch_size_constraints())  # Print the batch size constraints

import time
start = time.time()
x = sol.get_cost()
end = time.time()
print("Total Cost:", x )  # Print the total cost
print('Time', end - start)

Total Cost: 294208800.0
Time 0.005637168884277344


In [ ]:
from copy import deepcopy
class SimulatedAnnealing():
    def __init__(self, problem_instance, initial_solution, temperature=1000, cooling_rate=0.2,number_iteration=100, alpha = 0.05):
        self.problem_instance = problem_instance
        self.current_solution = initial_solution
        self.temperature = temperature
        self.cooling_rate = cooling_rate
        self.best_solution = initial_solution
        self.number_iteration=number_iteration
        self.alpha = alpha

    def acceptance_probability(self, new_cost, current_cost, temperature):
        if new_cost < current_cost:
            return 1.0
        return np.exp((current_cost - new_cost) / temperature)
    
    def partial_shuffle(self, permutation):
        """
        Shuffle only an alpha fraction of the permutation.
        """
        n = len(permutation)
        num_to_shuffle = int(self.alpha * n)  # Calculate the number of elements to shuffle
        if num_to_shuffle == 0:  # If alpha is too small, do nothing
            return permutation

        indices_to_shuffle = np.random.choice(n, size=num_to_shuffle, replace=False)  # Indices to shuffle
        # fixed_indices = np.setdiff1d(np.arange(n), indices_to_shuffle)  # Indices to keep fixed

        shuffled_part = np.random.permutation(permutation[indices_to_shuffle])  # Shuffle only the selected part

        # Combine fixed and shuffled parts
        new_permutation = permutation.copy()
        new_permutation[indices_to_shuffle] = shuffled_part

        return new_permutation

    def shuffle_permutation_body(self, candidate_solution):
        """
        Partially shuffle the body permutation of the candidate solution.
        """
        candidate_solution.body = self.partial_shuffle(candidate_solution.body)

    def shuffle_permutation_assembly(self, candidate_solution):
        """
        Partially shuffle the assembly permutation of the candidate solution.
        """
        candidate_solution.assembly = self.partial_shuffle(candidate_solution.assembly)

    def shuffle_permutation_paint(self, candidate_solution):
        """
        Partially shuffle the paint shop entry and update the paint shop exit.
        """
        candidate_solution.paint_entry = self.partial_shuffle(candidate_solution.paint_entry)
        # candidate_solution.paint_exit = 
        candidate_solution.get_paint_exit()

    def choose_candidate(self, candidate_solution):
        """
        Randomly choose a partial shuffle operation to apply to the candidate solution.
        """
        # The randomness could change the three at the same time !!!
        if  rd.random() < 1/3:
            self.shuffle_permutation_body(candidate_solution)
        if 1/3 <= rd.random()  < 2/3:
            self.shuffle_permutation_assembly(candidate_solution)
        if  rd.random() < 1/6:
            self.shuffle_permutation_paint(candidate_solution)    

        return candidate_solution

                    

    def run(self):
        iteration=0
        while self.temperature > 1  :

            candidate_solution = deepcopy(self.current_solution)

            candidate_solution = self.choose_candidate(candidate_solution)

            current_cost = self.current_solution.get_cost()
            candidate_cost = candidate_solution.get_cost()
            best_cost = self.best_solution.get_cost()

            if(iteration%100==0 ):
              print(f'Best Cost: {best_cost:.5e}')
              print(f'Current Cost: {current_cost:.5e}')
              print(f'Candidate Cost: {candidate_cost:.5e}')
              print("temperature is :", self.temperature )

            if current_cost < best_cost:
                self.best_solution = deepcopy(self.current_solution)

            ap = self.acceptance_probability(candidate_cost, current_cost, self.temperature)

            # if current_cost > new_cost :
            #   self.current_solution = candidate_solution

            if ap > rd.random():
              self.current_solution = deepcopy(candidate_solution)

            if(iteration%self.number_iteration==0):
              self.temperature *= 1 - self.cooling_rate
            iteration+=1

        return self.best_solution
    

initial solution: 289435260.0


In [ ]:
my_problem = Problem(input_file)
my_solution = solution(my_problem)
# my_solution.random_heuristic_construction()


print(f'initial solution: {my_solution.get_cost()}')

In [111]:
def find_best_solution(problem_instance, initial_solution, temperature = 600, cooling_rate=0.05 ,number_iteration=300,  alpha = 0.4): #5000 0.3 1000
            # temperature = (10**3)/np.log(2)
            sa = SimulatedAnnealing(problem_instance, initial_solution, temperature, cooling_rate,number_iteration , alpha=alpha)
            best_solution = sa.run()
            return best_solution


In [ ]:
best_solution=find_best_solution(my_problem,my_solution)
print(f'best solution: {best_solution.get_cost():.2e}')

Best Cost: 2.89435e+08
Current Cost: 2.89435e+08
Candidate Cost: 2.89435e+08
temperature is : 600
Best Cost: 2.87166e+08
Current Cost: 2.87166e+08
Candidate Cost: 2.87166e+08
temperature is : 570.0
Best Cost: 2.86856e+08
Current Cost: 2.86856e+08
Candidate Cost: 2.86856e+08
temperature is : 570.0
Best Cost: 2.86554e+08
Current Cost: 2.86554e+08
Candidate Cost: 2.86554e+08
temperature is : 570.0
Best Cost: 2.85977e+08
Current Cost: 2.85977e+08
Candidate Cost: 2.85977e+08
temperature is : 541.5
Best Cost: 2.85977e+08
Current Cost: 2.85977e+08
Candidate Cost: 2.87249e+08
temperature is : 541.5
Best Cost: 2.85950e+08
Current Cost: 2.85950e+08
Candidate Cost: 2.85950e+08
temperature is : 541.5
Best Cost: 2.85950e+08
Current Cost: 2.85950e+08
Candidate Cost: 2.86940e+08
temperature is : 514.425
Best Cost: 2.85950e+08
Current Cost: 2.85950e+08
Candidate Cost: 2.86270e+08
temperature is : 514.425
Best Cost: 2.85950e+08
Current Cost: 2.85950e+08
Candidate Cost: 2.85950e+08
temperature is : 514.

In [1]:
print('The best solution' ,best_solution.solution_project_format())

NameError: name 'best_solution' is not defined